# Part 1 : Extract 1 CV

In [1]:
# prompt: install tika
!pip install tika

In [2]:
# 2. Start the server (first import triggers Java download and setup)
from tika import parser
from tika import initVM
initVM()  # Explicitly initialize Java VM (optional but safer)


In [3]:
import json
import os
import requests


In [5]:
def extract_pdf_content(file_path):
    """
    Extract text content from a PDF file.

    Args:
        file_path (str): Path to the PDF file

    Returns:
        str: Extracted text content or None if extraction failed
    """
    parsed = parser.from_file(file_path)
    content = parsed['content']
    return content.strip() if content else None

pdf_content = extract_pdf_content("1-Amine_SAIGHI.pdf")
print(pdf_content)

2025-04-15 20:41:16,188 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/3.1.0/tika-server-standard-3.1.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/3.1.0/tika-server-standard-3.1.0.jar to /tmp/tika-server.jar.
2025-04-15 20:41:20,229 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/3.1.0/tika-server-standard-3.1.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/3.1.0/tika-server-standard-3.1.0.jar.md5 to /tmp/tika-server.jar.md5.
2025-04-15 20:41:21,316 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2025-04-15 20:41:26,321 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


AMINE 
SAIGHI 

Alger, Algérie   

+213 661 282 545   

amin.saighi@gmail.com   

Amine SAIGHI  

  

 RÉSUMÉ PROFESSIONNEL 
Un passionné de technologie, capable d'apprendre, de comprendre et de transformer les 
besoins métier en logiciels fiables et évolutifs conformes aux spécifications du client. 

 COMPÉTENCES 
• Languages: C#, ASP.net, TS 

• Base de donnée : SQL Server, MySQL 

• Service web: RESTful 

• Frameworks: .Net Core, Angular, ReactJs, 

ABP Framework 

• Design system: Ant-Design (NgZorro), 

Bootstrap, Material UI 

• Software: VS 2022, MS SQL Server, 

VSCode, Postman, Git, GitHub desktop 

• Fortes capacités de résolution de 

problems 

• Bonnes compétences en 

communication 

• Capacité d'apprentissage rapide 

 EDUCATION 
Master | en Ingénierie Logicielle | Université Constantine 2 
2012 – 2014 

License | en Ingénierie Logicielle | Université Constantine 2 
2008 – 2012 

 FORMATIONS 
DEVELOPPEMENT AGILE AVEC SCRUM | ITCOMP | Alger - 12/2021 
SCRUM | ITCOMP | Alg

In [5]:
import requests
import json
import os

def analyze_resume_with_qwen_json(resume_text, output_file="resume_data.json"):
    """
    Analyze a resume using Qwen and extract structured information in JSON format.
    Updated to use a simplified extracurricular format.

    Args:
        resume_text: Raw text extracted from the resume.
        output_file: Path to save the JSON output.

    Returns:
        Dictionary containing the structured resume information.
    """
    # Replace with your actual OpenRouter API key (preferably stored securely in environment variables)
    OPENROUTER_API_KEY = "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"

    # Correct endpoint for OpenRouter API
    API_ENDPOINT = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://your-site-url.com",  # Replace with your site URL
        "X-Title": "Resume Analyzer",      # Replace with your site name
    }

    prompt = f"""
You will receive a resume/CV in either English or French. Please extract all information and organize it into a structured JSON format.

Your response must be ONLY valid JSON with the following structure:
{{
  "personal_information": {{
    "name": "",
    "email": "",
    "phone": "",
    "address": "",
    "linkedin": ""
  }},
  "summary": [
    "Points from the summary or objective section"
  ],
  "education": [
    {{
      "degree": "",
      "institution": "",
      "period": ""
    }}
  ],
  "work_experience": [
    {{
      "position": "",
      "company": "",
      "period": "",
      "location": "",
      "description": ""
    }}
  ],
  "projects": [
    {{
      "name": "",
      "description": ""
    }}
  ],
  "skills": {{
    "category1": ["skill1", "skill2"],
    "category2": ["skill3", "skill4"]
  }},
  "languages": [
    {{
      "language": "",
      "proficiency": ""
    }}
  ],
  "certifications": [
    "certification1",
    "certification2"
  ],
  "publications": [
    "publication1",
    "publication2"
  ],
  "extracurricular": [
    "activity1",
    "activity2",
    "activity3"
  ]
}}

Important instructions:
- The content may be in English or French — keep the original language as is.
- Your entire response must be ONLY the JSON object, with no explanations or additional text.
- Include all sections in the JSON structure, even if they are empty.
- Extract all relevant details from the document accurately.
- Make sure the JSON is valid and properly formatted.
- For extracurricular activities, provide them as a simple list of strings, not as categories.

Resume Text:
{resume_text}
    """

    data = {
        "model": "qwen/qwen-2.5-72b-instruct",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "max_tokens": 5000
    }

    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for HTTP errors

        completion = response.json()
        content = completion["choices"][0]["message"]["content"]

        # Extract JSON content if it's wrapped in markdown code blocks
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].split("```")[0].strip()

        # Parse the JSON content
        resume_json = json.loads(content)

        # Ensure the output directory exists
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save the JSON data to a file with explicit error handling
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(resume_json, f, ensure_ascii=False, indent=2)
            print(f"Resume data successfully saved to {os.path.abspath(output_file)}")
        except Exception as save_error:
            print(f"Error saving file: {save_error}")
            # Try with a different path as fallback
            fallback_path = os.path.join(os.getcwd(), "resume_fallback.json")
            print(f"Trying fallback path: {fallback_path}")
            with open(fallback_path, 'w', encoding='utf-8') as f:
                json.dump(resume_json, f, ensure_ascii=False, indent=2)
            print(f"Resume data saved to fallback path: {fallback_path}")

        return resume_json
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"Status code: {e.response.status_code}")
            print(f"Response text: {e.response.text}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON parse error: {e}")
        print(f"Raw content received: {content if 'content' in locals() else 'No content available'}")

        # Try to extract JSON if it's in a malformed format
        try:
            # Find anything that looks like JSON
            import re
            json_pattern = r'\{.*\}'
            match = re.search(json_pattern, content, re.DOTALL)
            if match:
                fixed_json = match.group(0)
                resume_json = json.loads(fixed_json)

                # If we succeeded in parsing, save it
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(resume_json, f, ensure_ascii=False, indent=2)
                print(f"Fixed JSON saved to {output_file}")
                return resume_json
        except Exception as recovery_error:
            print(f"Recovery attempt failed: {recovery_error}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [ ]:
resume_json = analyze_resume_with_qwen_json(pdf_content, "output_resume.json")

Resume data successfully saved to /content/dehimi_yahia_resume.json


# Part 2 : Extract many Cvs and make it in one json file

**note :** you should run the function extract_pdf_content() from part 1 then run this

In [ ]:
# === Function to analyze a resume using Qwen ===
def analyze_resume_with_qwen_json(resume_text):
    OPENROUTER_API_KEY = "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"
    API_ENDPOINT = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://your-site-url.com",
        "X-Title": "Resume Analyzer",
    }

    prompt = f"""
You will receive a resume/CV in either English or French. Please extract all information and organize it into a structured JSON format.

Your response must be ONLY valid JSON with the following structure:
{{
  "personal_information": {{
    "name": "",
    "email": "",
    "phone": "",
    "address": "",
    "linkedin": ""
  }},
  "summary": [
    "Points from the summary or objective section"
  ],
  "education": [
    {{
      "degree": "",
      "institution": "",
      "period": ""
    }}
  ],
  "work_experience": [
    {{
      "position": "",
      "company": "",
      "period": "",
      "location": "",
      "description": ""
    }}
  ],
  "projects": [
    {{
      "name": "",
      "description": ""
    }}
  ],
  "skills": {{
    "category1": ["skill1", "skill2"],
    "category2": ["skill3", "skill4"]
  }},
  "languages": [
    {{
      "language": "",
      "proficiency": ""
    }}
  ],
  "certifications": [
    "certification1",
    "certification2"
  ],
  "publications": [
    "publication1",
    "publication2"
  ],
  "extracurricular": [
    "activity1",
    "activity2",
    "activity3"
  ]
}}

Important instructions:
- The content may be in English or French — keep the original language as is.
- Your entire response must be ONLY the JSON object, with no explanations or additional text.
- Include all sections in the JSON structure, even if they are empty.
- Extract all relevant details from the document accurately.
- Make sure the JSON is valid and properly formatted.

Resume Text:
{resume_text}
    """

    data = {
        "model": "qwen/qwen-2.5-72b-instruct",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 5000
    }

    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=data)
        response.raise_for_status()
        content = response.json()["choices"][0]["message"]["content"]

        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].split("```")[0].strip()

        return json.loads(content)

    except Exception as e:
        print(f"Error processing resume: {e}")
        return None


In [7]:
# === Function to process all resumes and create a single JSON file ===
def process_all_resumes(folder_path, output_file="all_resumes.json"):
    all_resumes_data = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            print(f"Processing: {filename}")

            try:
                text = extract_pdf_content(pdf_path)
                data = analyze_resume_with_qwen_json(text)
                if data:
                    ordered_data = {"file_name": filename}
                    ordered_data.update(data)  # Insert the rest of the JSON after the file name
                    all_resumes_data.append(ordered_data)
                else:
                    print(f"Failed to extract data from {filename}")
            except Exception as e:
                print(f"Exception while processing {filename}: {e}")

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_resumes_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ All resumes processed and saved to: {os.path.abspath(output_file)}")



Processing: 1-Amine_SAIGHI.pdf


KeyboardInterrupt: 

In [ ]:
# === MAIN ===
if __name__ == "__main__":
    folder_path = "/content"  # Replace with your actual folder name( i said folder of many CVs not file )
    process_all_resumes(folder_path)